In [ ]:
import torch
from pathlib import Path
from ibp_neurips19 import models
from ibp_neurips19.attacks import compute_robustness

In [ ]:
# model = 'small'
# dataset = 'CIFAR10'
# epsilon = 0.01
# net = models.fit_to_dataset(models.__dict__[f'{model}_cnn'](), dataset).eval()
# # checkpoint = torch.load(Path.home() / f'results/{dataset}-{model}_cnn-{epsilon}/0.001/checkpoint.pth')
# checkpoint = torch.load(Path.home() / f'results/dm_torch/{dataset.lower()}-{model}-{epsilon}.pth')
# net.load_state_dict(checkpoint['state_dict'])
# # 100 - checkpoint['best_acc1']

In [ ]:
# results = compute_robustness(net, dataset, 'cuda', restarts=1, subset=None, subset_seed=None, attack_kwargs=dict(epsilon=2/255))
# print(results.robustness)
# results.fooling_rate * 100

In [ ]:
from itertools import product 

main_dir = '/home/modar/results'
def experiments():
    datasets = ['MNIST', 'CIFAR10']
    epsilons = [0.01, 0.03, 0.1, 0.2]
    model_size = ['small', 'medium']
    learning_rates = [0, 1e-1, 1e-2, 1e-3]
    test_epsilons = [2 / 255, 0.1, 0.2, 0.3]
    for dataset, epsilon, model, learning_rate, test_epsilon in product(
            datasets, epsilons, model_size, learning_rates, test_epsilons):
        if dataset == 'CIFAR10':
            if test_epsilon != 2 / 255 or epsilon != 0.01:
                continue
        elif test_epsilon == 2 / 255 or epsilon == 0.01:
            continue
        yield dataset, epsilon, model, learning_rate, test_epsilon

for i, (dataset, epsilon, model, learning_rate,
        test_epsilon) in enumerate(experiments()):
    if model != 'medium' or dataset != 'CIFAR10' or epsilon != 0.01:
        continue
    if learning_rate == 0:
        checkpoint_file = (f'{main_dir}/dm_torch/'
                           f'{dataset.lower()}-{model}-{epsilon}.pth')
    else:
        checkpoint_file = (f'{main_dir}/{dataset}-{model}_cnn-{epsilon}'
                           f'/{learning_rate}/checkpoint.pth')
    checkpoint_file = Path(checkpoint_file)
#     print(i, f'{dataset}-{model}-{epsilon}', learning_rate, test_epsilon)
    checkpoint = torch.load(checkpoint_file)
    accuracy = checkpoint['accuracy'] if learning_rate == 0 else checkpoint['best_acc1'] / 100
    desired_test_epsilon = 2 / 255
    if test_epsilon != desired_test_epsilon:
        continue
    test_epsilon = checkpoint['PGD'][0]['epsilon']
    if test_epsilon != desired_test_epsilon:
        continue
    fooling_rate = checkpoint['PGD'][0]['fooling_rate']
    exp = f'{dataset}-{model}-{epsilon}:{test_epsilon}-{learning_rate}'
    print(f"{exp} [{100 * accuracy:0.2f}%] = {100 * fooling_rate:.2f}%")
    